In [24]:
from temple.planner import TUPlanner

planner = TUPlanner(username="", password="")
planner.select_term(202503)

In [ ]:
from util.print import pprint
from temple.classes import CourseSection

planner.select_courses(["MATH2043", "IH0851", "CIS1068", "EES2001", "MATH2101"])
planner.filter_sections({
    "MATH2043" : ["001", "002"],
    "EES2001" : ["001"],
    "CIS1068" : ["001", "002", "003"],
})
planner.set_filter(lambda c: c.instructionalMethod == "OLL" if c.subjectCourse == "IH0851" else True)
schedule_combos = planner.get_combinations(waitlist=True)

# pprint(schedule_combos)
pprint(len(schedule_combos))


In [ ]:
from temple.classes import CourseSection
from temple.week_schedule import WeekSchedule, Day
from util.colors import get_dark_mode_colors
from typing import Dict, List, Tuple

from IPython.display import Markdown, display
from ratemyprofessor.database import Teacher
from datetime import time
import matplotlib.pyplot as plt
import math


class SemesterSchedule:
    _time_slot: Dict[WeekSchedule, CourseSection]

    def __init__(self, _courses: List[CourseSection]):
        self._time_slot = {course.get_schedule(): course for course in _courses}

    def get_range(self) -> Tuple[float, float]:
        """Returns the minimum and maximum times of events in the schedule."""
        if not self._time_slot:
            return 0, 0

        time_min = 24
        time_max = 0

        for schedule in self._time_slot.keys():
            for time_range in schedule:
                time_min = min(time_min, time_range.start.to_hours())
                time_max = max(time_max, time_range.end.to_hours())
        return time_min, time_max

    def show(self, *, title: str = "Semester Schedule"):
        """Displays the weekly schedule using matplotlib."""
        plt.style.use("dark_background")
        plt.figure(figsize=(20, 8))

        day_label = [day.capitalize() for day in Day.names()]
        time_label = [time(hour % 24, 0).strftime("%I:%M %p") for hour in range(24, -1, -1)]
        # time_label = [f"{hour}:00" for hour in range(24, -1, -1)]

        plt.xticks(range(len(day_label)), day_label)
        plt.yticks(range(len(time_label)), time_label)
        plt.title(title)
        plt.tick_params(axis="both", which="major", pad=15)

        dark_mode_colors = get_dark_mode_colors()
        event_colors: Dict[str, str] = {}

        range_min, range_max = self.get_range()
        range_min = 24 - math.floor(range_min) + 0.25
        range_max = 24 - math.ceil(range_max) - 0.25
        plt.ylim(range_max, range_min)
        # plt.ylim(0, 24)

        for schedule, course in self._time_slot.items():
            for time_range in schedule:
                if course.subjectCourse not in event_colors:
                    event_colors[course.subjectCourse] = dark_mode_colors.pop(0)

                day_index = time_range.start.day.value
                class_color = event_colors[course.subjectCourse]

                beg: float = 24 - time_range.start.to_hours()
                end: float = 24 - time_range.end.to_hours()

                cell_width = 0.98
                l_side = day_index - cell_width / 2 + 0.02
                r_side = day_index + cell_width / 2 - 0.02
                t_padding = beg - 0.05
                b_padding = end + 0.05
                middle = (beg + end) / 2

                # Class time range
                plt.bar(day_index, end - beg, cell_width, beg, color=class_color, edgecolor="black")
                # Course title middle
                plt.text(day_index, middle, course.courseTitle, ha="center", va="center", fontsize=8)

                # Start time formatted top left
                plt.text(l_side, t_padding, time_range.start.format(), ha="left", va="top", fontsize=8)
                # End time formatted bottom left
                plt.text(l_side, b_padding, time_range.end.format(), ha="left", va="bottom", fontsize=8)
                # Section number top right
                plt.text(r_side, t_padding, f"Section: {course.sequenceNumber}", ha="right", va="top", fontsize=8)
                # Subject course buttom right
                plt.text(r_side, b_padding, course.subjectCourse, ha="right", va="bottom", fontsize=8)
        plt.show()

    def print_stats(self):
        def hm_fmt(_hours):
            # Calculate total seconds
            total_seconds = _hours * 3600

            # Calculate hours, minutes, and seconds
            h = int(total_seconds // 3600)
            m = int((total_seconds % 3600) // 60)

            return f"{h} hr{"s" if h != 1 else ""} {m} min{"s" if m != 1 else ""}"

        def render_table(_headers: List[str], _rows: List[List[str]]):
            # Create the header row
            md = "| " + " | ".join(_headers) + " |\n"
            md += "| " + " | ".join(["---"] * len(_headers)) + " |\n"  # Separator row

            # Add each row of data
            for row in _rows:
                # Replace newline characters with <br> for Markdown rendering
                sanitized_row = [cell.replace("\n", "<br>") for cell in row]
                md += "| " + " | ".join(sanitized_row) + " |\n"

            display(Markdown(md))

        print(
            f"WEEK_RANGE({hm_fmt(ScheduleCompare.week_range(self) / 60)}), "
            f"WEEK_TOTAL({hm_fmt(ScheduleCompare.week_total(self) / 60)}), "
            f"BREAK_TOTAL({hm_fmt(ScheduleCompare.between_total(self) / 60)})"
        )

        rows = [
            (
                course.subjectCourse,
                course.sequenceNumber,
                ("\n").join([teacher.get_name() for teacher in course.get_teachers()]),
                ("\n").join([f"{teacher.avgRatingRounded:.2f}" for teacher in course.get_teachers()]),
                "In-person" if course.instructionalMethod == "CLAS" else "Online",
                str(course.creditHourLow),
            )
            for course in sorted(self._time_slot.values(), key=lambda c: c.subjectCourse)
        ]
        render_table(["Class", "Section", "Teachers", "Ratings", "Type", "Credits"], rows)

        print(f"OVERALL_RATING: {ScheduleCompare.teacher_rating(self):.3f} avg")


class ScheduleCompare:
    """Class representing a function used to compare schedules for sorting."""

    def week_range(_s: SemesterSchedule):
        min, max = _s.get_range()
        return int((max - min) * 60)

    def week_total(_s: SemesterSchedule):
        total_time = 0
        for schedule in _s._time_slot.keys():
            for time_range in schedule:
                total_time += time_range.end.total_minutes() - time_range.start.total_minutes()
        return total_time

    def between_total(_s: SemesterSchedule):
        ranges_by_day = {}

        for schedule in _s._time_slot.keys():
            for time_range in schedule:
                assert time_range.start.day == time_range.end.day, "Must start and end on the same day"

                # Collect all time ranges by day
                ranges_by_day.setdefault(time_range.start.day, []).append(time_range)

        total_time = 0

        # Calculate the time between classes for each day
        for time_ranges in ranges_by_day.values():
            # Sort time ranges by start time
            time_ranges.sort(key=lambda x: x.start)

            # Calculate time between consecutive classes
            for i in range(len(time_ranges) - 1):
                end_of_current = time_ranges[i].end
                start_of_next = time_ranges[i + 1].start
                total_time += start_of_next.total_minutes() - end_of_current.total_minutes()

        return total_time

    def teacher_rating(_s: SemesterSchedule, *, penalty_rating=0.0, penalty_num_ratings=100.0):
        found_class = set()

        sum_rating = 0
        sum_num_ratings = 0

        for course in _s._time_slot.values():
            if course.subjectCourse not in found_class:
                found_class.add(course.subjectCourse)

                teachers = course.get_teachers()
                if len(teachers) == 0:
                    # Penalty for not having a rating
                    sum_rating += penalty_rating * penalty_num_ratings
                    sum_num_ratings += penalty_num_ratings
                else:
                    for teacher in teachers:
                        sum_rating += teacher.avgRatingRounded * teacher.numRatings
                        sum_num_ratings += teacher.numRatings
        return sum_rating / sum_num_ratings







schedules = [SemesterSchedule(schedule) for schedule in schedule_combos]
schedules = sorted(schedules, key=lambda s: (100000 - ScheduleCompare.teacher_rating(s), ScheduleCompare.between_total(s)))
schedules = list(schedules)[:50]

print(len(schedules))

for index, schedule in enumerate(schedules):
    schedule.print_stats()
    schedule.show(title=f"Semester Schedule {index + 1}")